In [ ]:
!pip install datasets sentencepiece

In [ ]:
import pandas as pd
from transformers import AutoTokenizer

In [ ]:
!pip install amseg

In [ ]:
!pip install pdf2image

In [ ]:
!pip install pytesseract

if you want to play with GPT token playground to see how it tokenize words and sentences [GPT token playground](https:/gpt-tokenizer.dev/)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open('/content/drive/MyDrive/10academy/labeled_telegram_product_price_location.txt', 'r') as file:
    lines = file.readlines()

# Process lines as needed
data = [line.strip().split('\t') for line in lines]  # Adjust the split based on your delimiter
df = pd.DataFrame(data)

In [ ]:
df

,0
0,Baby B-PRODUCT
1,bank I-PRODUCT
2,feeding I-PRODUCT
3,bottle I-PRODUCT
4,ዋጋ፦ I-PRICE
...,...
156672,ስላጋሩ O
156673,እናመሠግናለን O
156674,ቴሌግራም O
156675,ቻናል O


In [ ]:
# Process lines: separate tokens and labels
data = [line.strip().split() for line in lines if line.strip()]  # Split based on spaces
tokens = [item[0] for item in data]  # Extract tokens
labels = [item[1] for item in data]  # Extract labels

In [ ]:
# data

## Usinng different Amh tokenizers from HuggingFace



In [ ]:
# Initialize the tokenizer
model_name = "xlm-roberta-base"  # Change to any approprate model from Hugging Face if needed
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# Function to tokenize and align labels
def tokenize_and_align_labels(tokenizer, tokens, labels):
    aligned_tokens = []
    aligned_labels = []

    # Tokenize each word/token with associated label
    for word, label in zip(tokens, labels):
        tokenized_word = tokenizer.tokenize(word)  # Tokenize the word
        aligned_tokens.extend(tokenized_word)  # Add tokens to the list

        # Assign the label to the first subtoken and 'O' to subsequent subtokens
        aligned_labels.extend([label] + ['O'] * (len(tokenized_word) - 1))

    return aligned_tokens, aligned_labels

In [ ]:
# Tokenize and align labels
aligned_tokens, aligned_labels = tokenize_and_align_labels(tokenizer, tokens, labels)

In [ ]:
# Display the first 10 results
for token, label in zip(aligned_tokens[:20], aligned_labels[:20]):
    print(f"{token:20} {label}")

▁Baby                B-PRODUCT
▁bank                I-PRODUCT
▁feed                I-PRODUCT
ing                  O
▁bottle              I-PRODUCT
▁ዋጋ                  I-PRICE
፦                    O
▁1                   O
,                    O
900                  O
▁ብር                  I-PRICE
▁በ                   O
ቴ                    O
ሌ                    O
ግራ                   O
ም                    O
▁ለማ                  O
ዘ                    O
ዝ                    O
▁አድራሻ                O


### Using betterTokenizer which google's sentence piece





link for the tokenizer: [link](https://github.com/uhh-lt/amharicprocessor)

In [ ]:
from amseg.amharicSegmenter import AmharicSegmenter


In [ ]:
# Initialize the Amharic segmenter
sent_punct = []
word_punct = []
segmenter = AmharicSegmenter(sent_punct, word_punct)

In [ ]:
# Process lines: separate tokens and labels
data = [line.strip().split() for line in lines if line.strip()]  # Split based on spaces
tokens = [item[0] for item in data]  # Extract tokens
labels = [item[1] for item in data]  # Extract labels

In [ ]:
# Function to align tokens with their respective labels
def align_tokens_with_labels(tokenizer, tokens, labels):
    aligned_tokens = []
    aligned_labels = []

    # Tokenize each word/token with associated label
    for word, label in zip(tokens, labels):
        tokenized_word = tokenizer.amharic_tokenizer(word)  # Tokenize the word
        aligned_tokens.extend(tokenized_word)  # Add tokens to the list

        # Assign the label to the first subtoken and 'O' to subsequent subtokens
        aligned_labels.extend([label] + ['O'] * (len(tokenized_word) - 1))

    return aligned_tokens, aligned_labels

In [ ]:
# Example usage with different variable names
new_tokens, new_labels = align_tokens_with_labels(segmenter, tokens, labels)

In [ ]:
# Output the first 20 aligned tokens and labels
for token, label in zip(new_tokens[:20], new_labels[:20]):
    print(f"{token}: {label}")

Baby: B-PRODUCT
bank: I-PRODUCT
feeding: I-PRODUCT
bottle: I-PRODUCT
ዋጋ: I-PRICE
፦: O
1,900: O
ብር: I-PRICE
በቴሌግራም: O
ለማዘዝ: O
አድራሻችን: O
:-: O
1.መገናኛ: I-LOC
ኪኔሬት: O
ሕንፃ: O
1ኛ: O
ፎቅ: O
ቁ.: O
101: I-PRICE
(ንግድ: O


In [ ]:
# Saving the final tokens and labels to a CSV file
output_df = pd.DataFrame({'Token': new_tokens, 'Label': new_labels})
output_df.to_csv('/content/drive/MyDrive/10academy/final_tokens_labels.csv', index=False)

In [ ]:

# # Display results from index 50 to 80 with formatting
# for token, label in zip(aligned_tokens[:80], aligned_labels[:80]):
#     print(f"{token:<20} {label}")



```
# This is formatted as code
```

## Custom Amh Tokenizer

## Amharic SentencePiece Tokenizer

The **Amharic SentencePiece Tokenizer** segments Amharic text into subwords, enhancing the efficiency of natural language processing tasks like Named Entity Recognition (NER). By breaking down words into smaller units, it improves the model's ability to handle rare words and morphological variations.


Dataset used for training the custom tokenizer [dataset link](https://huggingface.co/datasets/israel/Amharic-News-Text-classification-Dataset?row=23)

In [ ]:
from datasets import load_dataset
import sentencepiece as spm

In [ ]:
#Load the dataset
dataset = load_dataset("israel/Amharic-News-Text-classification-Dataset")

README.md:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/150M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/37.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/41186 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10297 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'headline', 'category', 'date', 'views', 'article', 'link', 'word_len'],
        num_rows: 41186
    })
    test: Dataset({
        features: ['Unnamed: 0', 'headline', 'category', 'date', 'views', 'article', 'link', 'word_len'],
        num_rows: 10297
    })
})

In [ ]:
# Extract the text from the dataset
texts = dataset['train']['article']


In [ ]:
# texts

In [ ]:
# Filter out None values and prepare the data for SentencePiece
filtered_texts = [text for text in texts if text is not None]  # Remove None values
text_data = "\n".join(filtered_texts)

In [ ]:
# Save to a temporary file
with open('temp_text.txt', 'w', encoding='utf-8') as f:
    f.write(text_data)

In [ ]:
# Train SentencePiece model
spm.SentencePieceTrainer.train('--input=temp_text.txt --model_prefix=AmharicSPM --vocab_size=100000')

In [ ]:
# spm.SentencePieceTrainer.train('--input=temp_text.txt --model_prefix=am-word --model_type=word  --vocab_size=100000')


In [ ]:
tokenizer = spm.SentencePieceProcessor(model_file='AmharicSPM.model')

In [ ]:
# Tokenize the input text
text = "ቁ.2 ለቡ  መዳህኒዓለም ቤተ/ክርስቲያን ፊት ለፊት  #ዛም_ሞል 2ኛ ፎቅ ቢሮ ቁጥር.214"
tokens = tokenizer.encode(text, out_type=str)  # Use encode method for tokenization
#Print the tokenized words
print(tokens)

['▁ቁ', '.', '2', '▁ለቡ', '▁መዳ', 'ህ', 'ኒ', 'ዓ', 'ለም', '▁ቤተ', '/', 'ክርስቲያን', '▁ፊት', '▁ለፊት', '▁', '#', 'ዛ', 'ም', '_', 'ሞል', '▁2', 'ኛ', '▁ፎቅ', '▁ቢሮ', '▁ቁጥር', '.', '214']


### Using the custom tokenizer

In [ ]:
# Process lines: separate tokens and labels
data = [line.strip().split() for line in lines if line.strip()]  # Split based on spaces
tokens = [item[0] for item in data]  # Extract tokens
labels = [item[1] for item in data]  # Extract labels

In [ ]:
tokenizer = spm.SentencePieceProcessor(model_file='AmharicSPM.model')

In [ ]:
def tokenize_and_align_labels(tokenizer, tokens, labels):
    aligned_tokens = []
    aligned_labels = []
    token_ids = []

    for word, label in zip(tokens, labels):
        tokenized_ids = tokenizer.encode(word, out_type=int)  # Get token IDs
        tokenized_words = tokenizer.decode(tokenized_ids).split()  # Decode back to words

        aligned_tokens.extend(tokenized_words)  # Add decoded words to the list
        token_ids.extend(tokenized_ids)  # Add token IDs to the list

        aligned_labels.extend([label] + ['O'] * (len(tokenized_words) - 1))

    return aligned_tokens, token_ids, aligned_labels


In [ ]:
# Tokenize and align labels
custom_tokens, custom_ids, custom_labels = tokenize_and_align_labels(tokenizer, tokens, labels)


In [ ]:
# Display results from index 50 to 80 with tokenized words, their IDs, and labels
for token, token_id, label in zip(custom_tokens[:20], custom_ids[:20], custom_labels[:20]):
    print(f"{token:<25} {token_id:<10} {label}")

Baby                      34317      B-PRODUCT
bank                      56781      I-PRODUCT
feeding                   56768      I-PRODUCT
bottle                    3637       I-PRODUCT
ዋጋ                        52216      I-PRICE
⁇                         810        O
1                         559        O
⁇                         5957       O
900                       34395      O
ብር                        442        I-PRICE
በቴሌግራም                    0          O
ለማዘዝ                      171        O
አድራሻችን                    0          O
⁇                         6233       O
1                         54         I-LOC
⁇                         48679      O
መገናኛ                      15         O
ኪኔሬት                      23793      O
⁇                         9595       O
ንፃ                        911        O


In [ ]:
print(len(custom_tokens))  # Outputs the total number of tokens

208149


In [ ]:
# Assuming you have custom_tokens, custom_ids, and custom_labels as lists
with open("profoundly_confused_output.conll", "w", encoding="utf-8") as f:
    for token, token_id, label in zip(custom_tokens, custom_ids, custom_labels):
        f.write(f"{token}\t{token_id}\t{label}\n")
    f.write("\n")  # Adds a final newline to indicate the end